# Programa de Especialización en Procesamiento de Lenguaje Natural (NLP) 
## Tema: Procesamiento de Lenguaje Multilingüe y Traducción

#### Manuel Sigüeñas, M.Sc.(c)
#### Prof. Inteligencia artificial & Ciencia de Datos
[msiguenas@sdc-consulting-ti.com](msiguenas@sdc-consulting-ti.com)

### Desafíos del procesamiento de lenguaje multilingüe

Variabilidad morfológica y sintáctica: lenguas como el alemán o el chino presentan desafíos en la segmentación y estructura.

Idiomas con recursos limitados: falta de datos de entrenamiento para muchos idiomas.

Polisemia y contexto: palabras con múltiples significados según el idioma.

### Modelos de Traducción Automática

**MarianMT para traducción multilingüe**

Evolución de los modelos de traducción automática:

    - Modelos estadísticos basados en alineación de frases.
    - Modelos neuronales basados en RNN y GRU: avances y limitaciones.
    - Transformers: introducción a la atención y autoatención.

Modelos modernos:

    - MarianMT para traducción multilingüe.

URL: https://huggingface.co/Helsinki-NLP/opus-mt-en-es

- Instalar Sentencepiece

In [5]:
pip install sentencepiece

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/991.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/991.5 kB ? eta -:--:--
   -- ------------------------------------ 71.7/991.5 kB 777.7 kB/s eta 0:00:02
   --- ----------------------------------- 92.2/991.5 kB 581.0 kB/s eta 0:00:02
   ----- -------------------------------- 153.6/991.5 kB 762.6 kB/s eta 0:00:02
   ---------- ----------------------------- 256.0/991.5 kB 1.0 MB/s eta 0:00:01
   ------------- -------------------------- 337.9/991.5 kB 1.2 MB/s eta 0:00:01
   ----------------- ---------------------- 430.1/991.5 kB 1.3 MB/s eta 0:00:01
   ----------------- ---------------------- 430.1/991.5 kB 1.3 MB/s eta 0:00:01
   ---------------------- ----------------- 553.0/991.5 kB 1.3 MB/s eta 0:00:01
   ----------------------- ---------------- 583.7/991.5 kB 1.3 MB/s eta 0:00:01
   ------------------------ --------------- 614.4/991.5 kB 1.2 MB

In [6]:
import transformers
transformers.__version__

'4.44.0'

In [3]:
from transformers import MarianMTModel, MarianTokenizer
import sentencepiece

In [2]:
# Cargar el modelo preentrenado de traducción
model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)



C:\Users\SOCIAL DATA\.conda\envs\env_nlp\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Texto a traducir
text = ["Machine translation is a fascinating field.", "How are you?"]

# Traducción
inputs = tokenizer(text,
                   return_tensors="pt",
                   padding=True,
                   truncation=True)
translated = model.generate(**inputs)
translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

print("Traducción:", translated_text)


Traducción: ['Traducción automática es un campo fascinante.', '¿Cómo estás?']


### Evaluación de Traducción Automática

In [8]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score

In [9]:
# Traducción de referencia y generada
reference = ["La traducción automática es fascinante."]
candidate = ["La traducción automática es fascinante."]
# Tokenización de las oraciones (listas de palabras)
reference_tokenized = [ref.split() for ref in reference]
candidate_tokenized = candidate[0].split()

In [10]:
candidate_tokenized

['La', 'traducción', 'automática', 'es', 'fascinante.']

**BLEU**

Precisión de n-gramas entre la traducción generada y la referencia.

1. BLEU (Bilingual Evaluation Understudy Score)
Cómo funciona:

    BLEU mide la precisión de n-gramas (grupos de palabras consecutivas) entre la traducción generada por la máquina y una o más traducciones de referencia.
    Introduce un mecanismo de penalización llamado brevity penalty para evitar traducciones artificialmente cortas.
    No considera el orden global o semántico, pero mide qué tan similares son los fragmentos (n-gramas) entre las traducciones.

Valores que toma:

    Rango: 0 a 1 (o 0 a 100 si se multiplica por 100).
    1.0 significa una traducción idéntica a la referencia.
    0.0 significa que no hay coincidencia en los n-gramas.
    Generalmente, un BLEU > 0.3 (30%) se considera razonable para sistemas de traducción automática.
    Valores intermedios: BLEU suele ser bajo en traducciones complejas o para lenguajes con grandes diferencias estructurales.

Ventajas:

    Es simple y fácil de implementar.
    Es eficaz para medir la exactitud en traducciones cortas y técnicas.

Limitaciones:

    No considera sinónimos ni reordenamientos válidos en la estructura gramatical.
    Tiende a penalizar traducciones creativas o aquellas que no coincidan exactamente con la referencia.

In [11]:
# Cálculo del BLEU Score
bleu = sentence_bleu(reference_tokenized, candidate_tokenized)
print(f"BLEU Score: {bleu}")

BLEU Score: 1.0


**METEOR** 

Incluye análisis semántico y flexibilidad en sinónimos.

2. METEOR (Metric for Evaluation of Translation with Explicit ORdering)
Cómo funciona:

    METEOR evalúa la traducción basada en:
    Precisión y Recall: ¿Cuántas palabras coinciden entre la traducción generada y la referencia?
    Análisis semántico: Incluye sinónimos y lematización (identificación de raíces de palabras).
    Flexibilidad estructural: Considera el reordenamiento de palabras si el significado se mantiene.
    Combina estas métricas con una fórmula ponderada.

Valores que toma:

    Rango: 0 a 1 (o 0 a 100 si se multiplica por 100).
    1.0 indica una coincidencia perfecta en precisión, recall y orden.
    Valores cercanos a 0.0 indican una traducción muy diferente de la referencia.
    Normalmente, METEOR tiene valores ligeramente más altos que BLEU porque es más flexible.

Ventajas:

    Más robusto que BLEU, ya que permite sinónimos y considera reordenamientos gramaticales.
    Evalúa aspectos semánticos y no solo coincidencias exactas.

Limitaciones:

    Es más lento de calcular debido al análisis semántico.
    Requiere recursos adicionales (por ejemplo, diccionarios de sinónimos).

In [12]:
# Cálculo del METEOR Score
meteor = meteor_score(reference_tokenized, candidate_tokenized)
print(f"METEOR Score: {meteor}")

METEOR Score: 0.996


### Aplicación Práctica: Implementación de un Modelo Básico de Traducción

In [16]:
pip install sacremoses

   ---------------------------------------- 0.0/897.5 kB ? eta -:--:--
   ------ --------------------------------- 143.4/897.5 kB 8.3 MB/s eta 0:00:01
   ------------- -------------------------- 307.2/897.5 kB 4.8 MB/s eta 0:00:01
   --------------- ------------------------ 337.9/897.5 kB 3.0 MB/s eta 0:00:01
   -------------------- ------------------- 450.6/897.5 kB 2.6 MB/s eta 0:00:01
   ---------------------- ----------------- 501.8/897.5 kB 2.2 MB/s eta 0:00:01
   -------------------------- ------------- 583.7/897.5 kB 2.2 MB/s eta 0:00:01
   --------------------------- ------------ 614.4/897.5 kB 1.9 MB/s eta 0:00:01
   ------------------------------ --------- 686.1/897.5 kB 1.9 MB/s eta 0:00:01
   ---------------------------------- ----- 768.0/897.5 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------  880.6/897.5 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 897.5/897.5 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use u

**Despliegue del Modelo**

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from flask import Flask, request, jsonify
from transformers import MarianMTModel, MarianTokenizer

# Inicializar el modelo y el tokenizador
model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Crear la aplicación Flask
app = Flask(__name__)

# Ruta principal para traducción
@app.route("/translate", methods=["POST"])
def translate():
    # Obtener el texto del cuerpo de la solicitud
    data = request.get_json()
    text = data.get("text", "")

    # Verificar si el texto está presente
    if not text:
        return jsonify({"error": "No text provided"}), 400

    # Tokenización y traducción
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Devolver el texto traducido
    return jsonify({"translated_text": translated_text})

# Ejecutar la aplicación
if __name__ == "__main__":
    app.run(host="127.0.0.1", port=8000, debug=True, use_reloader=False)


C:\Users\SOCIAL DATA\.conda\envs\env_nlp\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [17/Apr/2025 20:00:27] "POST /translate HTTP/1.1" 200 -


In [ ]:
http://127.0.0.1:8000/translate